In [1]:
import numpy as np
import pandas as pd
import scipy
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

import sqlalchemy
from sqlalchemy import create_engine
import pymysql

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt


from six import StringIO
from IPython.display import Image 
from sklearn.tree import export_graphviz
import pydotplus
import graphviz

In [2]:
df = pd.read_csv('trainingData.csv', index_col=False)

In [3]:
# drop unnecessary, empty columns
df = df.drop(['Unnamed: 529','Unnamed: 530'],axis=1)

In [ ]:
# make sure all are int64
df.dtypes

In [4]:
df = df[df['BUILDINGID'] == 2]

In [5]:
df.head()

,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP,WAP001,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
46,-7331.748000,4864767.318,3,2,247,2,2,23,1371716690,100,...,100,100,100,100,100,100,-93,100,100,100
47,-7331.400500,4864768.479,3,2,248,2,2,23,1371716637,100,...,100,100,100,100,100,100,-89,100,100,100
49,-7327.492165,4864766.669,3,2,247,1,2,23,1371716719,100,...,100,100,100,100,100,100,100,-91,100,100
50,-7336.700400,4864764.479,3,2,246,2,2,23,1371716762,100,...,100,100,100,100,100,100,-90,100,100,100
51,-7337.394800,4864763.227,3,2,245,2,2,23,1371716797,100,...,100,100,100,100,100,100,-92,100,100,100


In [6]:
df = df.drop(columns=['USERID','PHONEID','LONGITUDE','LATITUDE','TIMESTAMP'])

In [8]:
#recode to string
df = df.astype(str)
df.head

<bound method NDFrame.head of       FLOOR BUILDINGID SPACEID RELATIVEPOSITION WAP001 WAP002 WAP003 WAP004  \
46        3          2     247                2    100    100    100    100   
47        3          2     248                2    100    100    100    100   
49        3          2     247                1    100    100    100    100   
50        3          2     246                2    100    100    100    100   
51        3          2     245                2    100    100    100    100   
...     ...        ...     ...              ...    ...    ...    ...    ...   
19926     1          2     206                2    100    100    100    100   
19927     1          2     141                2    100    100    100    100   
19928     1          2     207                2    100    100    100    100   
19931     1          2     140                1    100    100    100    100   
19933     1          2     140                2    100    100    100    100   

      WAP005 WAP006  

In [9]:
# IV
LOCATE = df['FLOOR'] + df['BUILDINGID'] + df['SPACEID'] + df['RELATIVEPOSITION']

In [10]:
LOCATE = LOCATE.astype(np.int64)

In [11]:
df.astype(np.int64)

,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
46,3,2,247,2,100,100,100,100,100,100,...,100,100,100,100,100,100,-93,100,100,100
47,3,2,248,2,100,100,100,100,100,100,...,100,100,100,100,100,100,-89,100,100,100
49,3,2,247,1,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-91,100,100
50,3,2,246,2,100,100,100,100,100,100,...,100,100,100,100,100,100,-90,100,100,100
51,3,2,245,2,100,100,100,100,100,100,...,100,100,100,100,100,100,-92,100,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19926,1,2,206,2,100,100,100,100,100,100,...,100,100,100,100,100,100,-87,100,100,100
19927,1,2,141,2,100,100,100,100,100,100,...,100,100,100,100,100,-84,-77,100,100,100
19928,1,2,207,2,100,100,100,100,100,100,...,100,100,100,100,100,-90,-87,100,100,100
19931,1,2,140,1,100,100,100,100,100,100,...,100,100,100,100,100,100,-91,100,100,100


In [12]:
# create DV
df = df.drop(columns=['FLOOR','BUILDINGID','SPACEID','RELATIVEPOSITION'])

In [13]:
df.head()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP511,WAP512,WAP513,WAP514,WAP515,WAP516,WAP517,WAP518,WAP519,WAP520
46,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-93,100,100,100
47,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-89,100,100,100
49,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,-91,100,100
50,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-90,100,100,100
51,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,-92,100,100,100


In [14]:
oobY = LOCATE

In [15]:
oobX = df

In [16]:
X_trainOOB, X_testOOB, y_trainOOB, y_testOOB = train_test_split(oobX, oobY, test_size = .25, random_state = 123)   
print(X_trainOOB.shape)
print(y_trainOOB.shape)
print(X_testOOB.shape)
print(y_testOOB.shape)

(7119, 520)
(7119,)
(2373, 520)
(2373,)


In [17]:
algos = []
algos.append(('Random Forest', RandomForestClassifier()))
algos.append(('Linear Regression',LinearRegression()))
algos.append(('Support Vector Regression',SVR()))
algos.append(('Decision Tree Classifier', DecisionTreeClassifier()))
algos.append(('KNeighbors Classifier', KNeighborsClassifier()))  

In [18]:
results = []
names = []

for name, model in algos:
    result = cross_val_score(model, X_trainOOB, y_trainOOB,  cv=3)
    names.append(name)
    results.append(result)

C:\Users\hey\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\hey\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\hey\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"


In [19]:
for i in range(len(names)):
    print(names[i],results[i].mean())

Random Forest 0.7859249894648125
Linear Regression 0.8862558912805573
Support Vector Regression -0.00797198214723053
Decision Tree Classifier 0.5177693496277567
KNeighbors Classifier 0.5632813597415367


In [20]:
bestAlgo = LinearRegression(n_jobs=4)
bestAlgoOOBpred = bestAlgo.fit(X_trainOOB, y_trainOOB).predict(X_testOOB)
print(classification_report(y_testOOB, bestAlgoOOBpred))
print(confusion_matrix(y_testOOB, bestAlgoOOBpred))

              precision    recall  f1-score   support

    -45920.0       0.00      0.00      0.00       0.0
    -45728.0       0.00      0.00      0.00       0.0
    -43424.0       0.00      0.00      0.00       0.0
    -42144.0       0.00      0.00      0.00       0.0
    -41504.0       0.00      0.00      0.00       0.0
    -35776.0       0.00      0.00      0.00       0.0
    -32448.0       0.00      0.00      0.00       0.0
    -30944.0       0.00      0.00      0.00       0.0
    -29696.0       0.00      0.00      0.00       0.0
    -27776.0       0.00      0.00      0.00       0.0
    -27648.0       0.00      0.00      0.00       0.0
    -27072.0       0.00      0.00      0.00       0.0
    -25632.0       0.00      0.00      0.00       0.0
    -23104.0       0.00      0.00      0.00       0.0
    -22720.0       0.00      0.00      0.00       0.0
    -20768.0       0.00      0.00      0.00       0.0
    -20608.0       0.00      0.00      0.00       0.0
    -18176.0       0.00    

C:\Users\hey\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hey\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hey\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hey\anaconda3\lib\site-packages\sklearn\metrics\_classificat

In [21]:
predictions = bestAlgo.predict(X_testOOB)
print(predictions)

[227264. 344320.  68672. ... 314240. 329760. 301024.]
